In [12]:
import ee
import numpy as np
import pandas as pd
import pickle
import datetime

In [13]:
ee.Authenticate()

Enter verification code:  4/1AX4XfWjOla0aggoxBUmdDmadlWZW2enpRDNLz3s-76YtCPThsnhhVjR5eZE



Successfully saved authorization token.


In [14]:
ee.Initialize()

In [15]:
# sampling paramaters
randomSeed = 109
collectionLandsatL1c = "LANDSAT/LC08/C02/T1_TOA" # landsat image collection
collectionLandsatL2a= "LANDSAT/LC08/C02/T1_L2" # landsat image collection
cloudCoverL0X = 10# maximum cloud cover of sampled Landsat images
maxSample = 10000 # maximum samples to extract from each Landsat image
outputScale = 500 # scale of output comparisons , increasing this will eventually exceed memory
subimageSize = 40000 # m dimension of subimage we will randomly extract from Landsat image to avoid memory limits
minfracAreaLC = 0.5 # minimum cloud free valid  Landsat subimage fraction
minfracAreaMCD15 = 0.25 # minimum cloud free valid MODIS area in Landsat subimage fraction's unmasked area
minfracAreaLCtoMCD = 0.10 # valid aggregated landsat fraction
minfracValidLC = 0.9 # minimum facation of valid landsat pixels in each aggregated modis pixels to estimate mean values
deltaRow = 1  # skip this many WRS rows when sampling Landsat scenes
deltaPath = 1 # skip this many WRS paths when sampling Landsat scenes
deltaDay = 15 # search this many contiguous days when looking for a cloud free Landsat scene
maxPixels = 300 # max landsat pixels allowed to be aggregated in an output pixel, should be about 20% larger than the ratio of (outputScale/30) (cannot exceed 1024)

#output files
InputFileNameB36 = '/home/richard_fernandes/LUTinputLC0XB36calv9'
InputFileNameB45 = '/home/richard_fernandes/LUTinputLC0XB45calv9'
InputFileNameMOD09 = '/home/richard_fernandes/LUTinputMOD09calv9'
InputFileNameMOD09bands = '/home/richard_fernandes/LUTinputMOD09bandscalv9'
InputFileNameLat = '/home/richard_fernandes/LUTinputLC0XLatcalv9'
InputFileNameLon = '/home/richard_fernandes/LUTinputLC0XLoncalv9'
OutputFileName = '/home/richard_fernandes/LUToutputLC0Xcalv9'
YearFileName = '/home/richard_fernandes/YearLC0Xcalv9'
MonthFileName = '/home/richard_fernandes/MonthLC0Xcalv9'
DayFileName = '/home/richard_fernandes/DayLC0Xcalv9'
PathFileName = '/home/richard_fernandes/PathLC0Xcalv9'
RowFileName = '/home/richard_fernandes/RowLC0Xcalv9'
FparLai_QCName = '/home/richard_fernandes/FparLai_QCcalv9'
FparExtra_QCName = '/home/richard_fernandes/FparExtra_QCcalv9'




In [16]:
# Match MOD09A1.006 daily reflectance data and MCD12Q1 annual biome type to  MCD15A3H.006 image composite
# masking any dates without exactly one MOD09A1 valid clear sky value and one valid MCD153AH terra retrieval
def matchMOD09A1toMCD15(imageMCD15):
    
    # get date range of MCD15A3Himage
    d1 = ee.Date(imageMCD15.get('system:time_start'));
    d2 = ee.Date(imageMCD15.get('system:time_end'));
    
#    print(d1.getInfo())
#    print(d2.getInfo())
    # find MOD09A1 products within date range of MCD15 and mask them 
    colMOD09A1 = ee.ImageCollection("MODIS/006/MOD09GA").filterDate(d1,d2).map(lambda image: image.updateMask(image.select('QC_500m').bitwiseAnd(1<<1).eq(0)) \
                                                                                                  .updateMask(image.select('state_1km').bitwiseAnd(1<<0).eq(0)) \
                                                                                                  .updateMask(image.select('state_1km').bitwiseAnd(1<<1).eq(0)) \
                                                                                                  .updateMask(image.select('state_1km').bitwiseAnd(1<<2).eq(0)) \
                                                                                                  .addBands(image.select('sur_refl_b02').divide(image.select('sur_refl_b01').add(1)).rename('SR_VI')))
                                                                                                 # .updateMask(image.select('state_1km').bitwiseAnd(1<<3).eq(1)) )
                                                                                                #  .updateMask(image.select('state_1km').bitwiseAnd(1<<4).eq(0)) \
                                                                                                #   .updateMask(image.select('state_1km').bitwiseAnd(1<<5).eq(0)) \
                                                                                               #   .updateMask(image.select('state_1km').bitwiseAnd(1<<6).gt(-1)) \
                                                                                              #    .updateMask(image.select('state_1km').bitwiseAnd(1<<7).eq(0)) \
                                                                                                #  .updateMask(image.select('state_1km').bitwiseAnd(1<<8).eq(8)) \
                                                                                                #  .updateMask(image.select('state_1km').bitwiseAnd(1<<9).eq(0)) \
                                                                                               #   .updateMask(image.select('state_1km').bitwiseAnd(1<<10).eq(0)) \
                                                                                                #  .updateMask(image.select('state_1km').bitwiseAnd(1<<11).eq(1)) \
                                                                                                 # .updateMask(image.select('state_1km').bitwiseAnd(1<<12).eq(0)) \
                                                                                                 # .updateMask(image.select('state_1km').bitwiseAnd(1<<13).eq(0)) \
                                                                                                 # .updateMask(image.select('state_1km').bitwiseAnd(1<<14).eq(0)) \
                                                                                                 # .updateMask(image.select('state_1km').bitwiseAnd(1<<15).eq(0)))
    #    print(colMOD09A1.size().getInfo())
    # get biome type map
    imageMCD12Q1 = ee.ImageCollection("MODIS/006/MCD12Q1").map(lambda image: image.set('daterange',ee.DateRange(ee.Date(image.get('system:time_start')),ee.Date(image.get('system:time_end')))))\
                                                            .filter(ee.Filter.dateRangeContains('daterange', d1)) \
                                                            .first() \
                                                            .select('LC_Type3') 
#    print(imageMCD12Q1.getInfo())

# make mask of pixels containing only one valid cloud free MOD09 500m retrieval
    # and a primary algorithm retyrieval from terra wigthout significant clouds
    # and add MOD09A1 composite created by apply the MCD15 mask to the MOOD9A1 collection and composite it
    # and add biome type
    imageMCD15 = imageMCD15.updateMask( colMOD09A1.count().select('sur_refl_b01').gt(-1) ) \
                            .updateMask( imageMCD15.select('FparLai_QC').bitwiseAnd(1<<0).eq(0)) \
                            .updateMask( imageMCD15.select('FparLai_QC').bitwiseAnd(1<<1).eq(0)) \
                            .updateMask( imageMCD15.select('FparLai_QC').bitwiseAnd(1<<2).eq(0)) \
                            .updateMask( imageMCD15.select('FparLai_QC').bitwiseAnd(1<<3).eq(0)) \
                            .updateMask( imageMCD15.select('FparLai_QC').bitwiseAnd(1<<4).eq(0)) \
                            .updateMask(imageMCD15.select('FparLai_QC').bitwiseAnd(1<<6).eq(0)) \
                            .updateMask(imageMCD15.select('FparLai_QC').bitwiseAnd(1<<7).eq(0)) \
                            .updateMask(imageMCD15.select('FparExtra_QC').bitwiseAnd(1<<1).eq(0)) \
                            .updateMask( imageMCD12Q1.select('LC_Type3').gt(0)) \
                            .addBands( imageMCD12Q1 )
    imageMCD15 = imageMCD15.addBands(colMOD09A1.map(lambda image: image.mask(imageMCD15.select('Lai'))).qualityMosaic('SR_VI'))


    
   
    
    
    return imageMCD15

In [17]:
# coded LUT for output MODIS MCD15
def getLUTout(imageMCD15):
    imageMCD15 = ee.Image(imageMCD15);

    return imageMCD15.select('Lai').min(99).multiply(1).toInt64() \
                        .add(imageMCD15.select('LaiStdDev').min(99).multiply(1e2).toInt64()) \
                        .add(imageMCD15.select('Fpar').min(99).multiply(1e4).toInt64()) \
                        .add(imageMCD15.select('FparStdDev').min(99).multiply(1e6).toInt64()) \
                        .add(imageMCD15.select('satFlag').multiply(100).multiply(1e9).toInt64()) \
                        .add(imageMCD15.select('snowFlag').multiply(100).multiply(1e12).toInt64()) \
                        .add(imageMCD15.select('LC_Type3').multiply(1e14).toInt64()) \
                        .rename('LUTout')

In [18]:
# Code LUT for an input MOD09 image , same as getLUTin but no satFlag
def getLUTinMOD09(imageMCD15MOD09):
    
     return imageMCD15MOD09.select('sur_refl_b01').add(100).max(0).min(9999).floor().toInt64().multiply(1).toInt64() \
                        .add(imageMCD15MOD09.select('sur_refl_b02').add(100).max(0).min(9999).floor().toInt64().multiply(1e4).toInt64()) \
                        .add(imageMCD15MOD09.select('SolarZenith').multiply(0.01).max(0).min(99).floor().toInt64().multiply(1e8).toInt64()) \
                        .add(imageMCD15MOD09.select('SolarAzimuth').multiply(0.01).add(180).max(0).min(360).floor().toInt64().multiply(1e10).toInt64()) \
                        .add(imageMCD15MOD09.select('SensorZenith').multiply(0.01).max(0).min(99).floor().toInt64().multiply(1e13).toInt64()) \
                        .add(imageMCD15MOD09.select('SensorAzimuth',).multiply(0.01).add(180).max(0).min(360).floor().toInt64().multiply(1e15).toInt64()) \
                        .rename('LUTinMOD09')

In [19]:
# Code LUT for an input MOD09 image , same as getLUTin but no satFlag
def getLUTinMOD09bands(imageMCD15MOD09):
    
     return imageMCD15MOD09.select('sur_refl_b03').add(100).max(0).min(9999).floor().toInt64().multiply(1).toInt64() \
                        .add(imageMCD15MOD09.select('sur_refl_b04').add(100).max(0).min(9999).floor().toInt64().multiply(1e4).toInt64()) \
                        .add(imageMCD15MOD09.select('sur_refl_b06').add(100).max(0).min(9999).floor().toInt64().multiply(1e8).toInt64()) \
                        .add(imageMCD15MOD09.select('sur_refl_b07').add(100).max(0).min(9999).floor().toInt64().multiply(1e12).toInt64()) \
                        .rename('LUTinMOD09bands')

In [20]:
# coded LUT for input landsat image red and nir band
def getLUTL0XinB45(imageMCD15):
    imageMCD15 = ee.Image(imageMCD15);
    return imageMCD15.select('SR_B4').multiply(2.75e-05).add(-0.2).max(0).min(0.999).multiply(1000).floor().multiply(1).toInt64() \
                        .add(imageMCD15.select('SR_B5').multiply(2.75e-05).add(-0.2).max(0).min(0.999).multiply(1000).floor().multiply(1e3).toInt64()) \
                        .add(imageMCD15.select('VZA').multiply(0.01).round().multiply(1e6).toInt64()) \
                        .add(imageMCD15.select('VAA').multiply(0.01).add(180).round().multiply(1e8).toInt64()) \
                        .add(imageMCD15.select('SZA').multiply(0.01).round().multiply(1e11).toInt64()) \
                        .add(imageMCD15.select('SAA').multiply(0.01).add(180).round().multiply(1e13).toInt64()) \
                        .rename('LUTL0XinB45')







In [21]:
# coded LUT for input landsat image green and swir
def getLUTL0XinB36(imageMCD15):
    imageMCD15 = ee.Image(imageMCD15);
    return imageMCD15.select('SR_B3').multiply(2.75e-05).add(-0.2).max(0).min(0.999).multiply(1000).floor().multiply(1).toInt64() \
                        .add(imageMCD15.select('SR_B6').multiply(2.75e-05).add(-0.2).max(0).min(0.999).multiply(1000).floor().multiply(1e3).toInt64()) \
                        .add(imageMCD15.select('VZA').multiply(0.01).round().multiply(1e6).toInt64()) \
                        .add(imageMCD15.select('VAA').multiply(0.01).add(180).round().multiply(1e8).toInt64()) \
                        .add(imageMCD15.select('SZA').multiply(0.01).round().multiply(1e11).toInt64()) \
                        .add(imageMCD15.select('SAA').multiply(0.01).add(180).round().multiply(1e13).toInt64()) \
                        .rename('LUTL0XinB36')


In [22]:
# sample worldwide 
# check each year and month
cumsam = 0
for year in range(2013,2022):#

        
    # specify land cover projected into MOD15 projection and aggregated using median to outputScale    
    projectionMCD15=  ee.ImageCollection('MODIS/006/MCD15A3H').first().select('Lai').projection()
    projectionMCD12 =  ee.ImageCollection("MODIS/006/MCD12Q1").first().select('LC_Type3').projection()
    imageMCD12Q1 = ee.ImageCollection("MODIS/006/MCD12Q1").filterDate(ee.Date.fromYMD(year,1,1),ee.Date.fromYMD(year,12,31)).first() \
                     .select('LC_Type3') \
                     .reproject(crs= projectionMCD15,crsTransform= None,scale=projectionMCD12.nominalScale() ) \
                     .reduceResolution(reducer= ee.Reducer.median(),maxPixels= 1024,bestEffort=False) \
                     .reproject(crs= projectionMCD15.atScale(outputScale))      

    # sample monthly data
    for month in [6,7,5,8,4,9,3,10,2,11,1,12]:  
        
        #sample WRS path rows
        for path in range(1,70,deltaPath): 
            for row in range(10,40,deltaRow): 
                
                # sample deltaDay intervals of each month
                for daystart in range (1,30-deltaDay,deltaDay):
                    
                    # find Landsat images for paht , row and date interval between 0 and maxCloudCover
                    imageLC = ee.ImageCollection(collectionLandsatL2a).filterMetadata('WRS_PATH','equals',path) \
                                                                          .filterMetadata('WRS_ROW','equals',row) \
                                                                          .filterDate(ee.Date.fromYMD(year,month,daystart),ee.Date.fromYMD(year,month,daystart).advance(deltaDay,'days')) \
                                                                          .filter(ee.Filter.lte('CLOUD_COVER_LAND',cloudCoverL0X))
                    
                    # only proceed if there is at least one filtered landsat image (we will only use the first one found)
                    if (imageLC.size().getInfo() > 0) : 

                        # only sample the first scene we found in the interval of deltaDay's 
                        imageLC = imageLC.first()
                        projectionLC = imageLC.select('QA_PIXEL').projection()
                        
                        # get date range of imageLC and print it out
                        d1 = ee.Date(imageLC.get('system:time_start'))
                        d2 = ee.Date(imageLC.get('system:time_end'))     
                        print('year',year,'month',month,'path',path,'row',row)

                        # get  MOD15 product containing the aquistion of the  Landsat image
                        imageMCD15 = ee.ImageCollection('MODIS/006/MCD15A3H') \
                                                    .map(lambda image: image.set('daterange',ee.DateRange(ee.Date(image.get('system:time_start')),ee.Date(image.get('system:time_end')))))\
                                                    .filter(ee.Filter.dateRangeContains('daterange', d1)) \
                                                    .map(lambda image: matchMOD09A1toMCD15(image) ) 

                        # only proceed if there is a MCD15 product containing the Landsat image aquisition
                        if (imageMCD15.size().getInfo() > 0)  :

                            
                            #randomly sample a sub-image to avoid memory issues, 
                            # add a validPixel we will use to determine aggregation and unmask since we will accept some missing data
                            imageGRID = imageLC.geometry().coveringGrid(projectionMCD15,subimageSize).randomColumn(columnName='randomIndex',seed=randomSeed)                                
                            imageLC = imageLC.addBands(ee.ImageCollection(collectionLandsatL1c) \
                                                                          .filterMetadata('system:index','equals',imageLC.get('system:index')) \
                                                                          .first() \
                                                                          .select(['VZA','VAA','SZA','SAA'])) \
                                             .clip(ee.Feature(imageGRID.sort('randomIndex').first()).geometry()) \
                                             .updateMask(imageLC.select('QA_PIXEL').bitwiseAnd(1<<0).eq(0)) \
                                             .updateMask(imageLC.select('QA_PIXEL').bitwiseAnd(1<<1).eq(0)) \
                                             .updateMask(imageLC.select('QA_PIXEL').bitwiseAnd(1<<2).eq(0)) \
                                             .updateMask(imageLC.select('QA_PIXEL').bitwiseAnd(1<<3).eq(0)) \
                                             .updateMask(imageLC.select('QA_PIXEL').bitwiseAnd(1<<4).eq(0)) 
        
                            
                            # get the fraction of area with unmasked LC data 
                            validArea = ee.Number(ee.Image.pixelArea().mask(imageLC.select('QA_PIXEL').mask()).reduceRegion(reducer=ee.Reducer.sum(),geometry=imageLC.geometry(),scale=30,bestEffort=True).get('area')).getInfo()
                            totalArea = ee.Number(ee.Image.pixelArea().mask(imageLC.select('QA_PIXEL').unmask().mask()).reduceRegion(reducer=ee.Reducer.sum(),geometry=imageLC.geometry(),scale=30,bestEffort=True).get('area')).getInfo()
                            
                            fracAreaLC = 0 
                            if (totalArea > 0) :
                                fracAreaLC = validArea / totalArea
                            #print('fracAreaLC:',fracAreaLC)  

                            # proceed if there is sufficient landsat area
                            if ( fracAreaLC > minfracAreaLC) :
                                                
                                # get MCD15 image from collection, clip to Landsat subimage
                                imageMCD15 = imageMCD15.first().clip(imageLC.geometry()) 
                                                
                                # update mask and add land cover and a saturation flag band
                                imageMCD15 = imageMCD15.updateMask( imageMCD15.select('FparLai_QC').bitwiseAnd(1<<0).eq(0)) \
                                                .updateMask( imageMCD15.select('FparLai_QC').bitwiseAnd(1<<2).eq(0)) \
                                                .updateMask( imageMCD15.select('FparLai_QC').bitwiseAnd(1<<3).eq(0)) \
                                                .updateMask( imageMCD15.select('FparLai_QC').bitwiseAnd(1<<4).eq(0)) \
                                                .updateMask( imageMCD15.select('FparLai_QC').bitwiseAnd(1<<6).eq(0)) \
                                                .updateMask( imageMCD15.select('FparLai_QC').bitwiseAnd(1<<7).eq(0)) \
                                                .addBands(imageMCD15.select('FparLai_QC').rename(['satFlag']).bitwiseAnd(1<<5).eq(1)) \
                                                .addBands(imageMCD15.select('FparExtra_QC').rename(['snowFlag']).bitwiseAnd(1<<2).eq(1)) \
                                                .updateMask( imageMCD15.select('FparExtra_QC').bitwiseAnd(1<<0).eq(0)) \
                                                .updateMask( imageMCD15.select('FparExtra_QC').bitwiseAnd(1<<1).eq(0)) \
                                                .updateMask( imageMCD15.select('FparExtra_QC').bitwiseAnd(1<<3).eq(0)) \
                                                .updateMask( imageMCD15.select('FparExtra_QC').bitwiseAnd(1<<4).eq(0)) \
                                                .updateMask( imageMCD15.select('FparExtra_QC').bitwiseAnd(1<<5).eq(0)) \
                                                .updateMask( imageMCD15.select('FparExtra_QC').bitwiseAnd(1<<6).eq(0)) \


                            
                                # get the fraction of area with unmasked MCD15 data 
                                validArea = ee.Number(ee.Image.pixelArea().mask(imageMCD15.select('Lai').mask()).reduceRegion(reducer=ee.Reducer.sum(),geometry=imageLC.geometry(),crs=projectionMCD15.atScale(outputScale)).get('area')).getInfo()
                                totalArea = ee.Number(ee.Image.pixelArea().mask(imageMCD15.select('Lai').unmask().mask()).reduceRegion(reducer=ee.Reducer.sum(),geometry=imageLC.geometry(),crs=projectionMCD15.atScale(outputScale)).get('area')).getInfo()
                                fracAreaMCD15 = 0 
                                if (totalArea > 0) :
                                    fracAreaMCD15 = validArea / totalArea
                                print('fracAreaMCD15:',fracAreaMCD15) 
                           
                                # sample if there is sufficient unmasked area
                                if ( fracAreaMCD15> minfracAreaMCD15 ) :
                                                
                                    
                                    # add a view zenith angle band and a validPixel band to the landsat image
                                    # then unmask and aggregate to output scale, we unmask first to allow for <100% validPixel s
                                    imageLCtoMCD = imageLC.addBands(imageLC.unmask().select('QA_PIXEL').gt(-1).rename('validPixel')) \
                                                         .reproject(crs= projectionMCD15,scale=projectionLC.nominalScale() ) \
                                                         .reduceResolution(reducer= ee.Reducer.mean(),maxPixels=maxPixels,bestEffort=False) \
                                                         .reproject(crs= projectionMCD15 , scale= outputScale) \
                                                         .updateMask(imageMCD15.select('FparLai_QC').mask())
                                   
                                    # mask bands not meeting fractional pixel threshold
                                    # we then have to divide by the fractional validPixel band to adjust for  zero masked values 
                                    validLCtoMCD = imageLCtoMCD.select('validPixel').gte(minfracValidLC);
                                    imageLCtoMCD = imageLCtoMCD.updateMask(validLCtoMCD)


                                    # add the aggregated LC image to the modis image and update the MCD image with the valid mask
                                    imageMCD15 = imageMCD15.addBands(imageLCtoMCD) \
                                                            .updateMask(validLCtoMCD) 
                                                            
                                    # get the fraction of area with unmasked LC data 
                                    validArea = ee.Number(ee.Image.pixelArea().mask(imageMCD15.select('QA_PIXEL').mask()).reduceRegion(reducer=ee.Reducer.sum(),geometry=imageMCD15.geometry(),scale=outputScale,bestEffort=True).get('area')).getInfo()
                                    totalArea = ee.Number(ee.Image.pixelArea().mask(imageMCD15.select('QA_PIXEL').unmask().mask()).reduceRegion(reducer=ee.Reducer.sum(),geometry=imageMCD15.geometry(),scale=outputScale,bestEffort=True).get('area')).getInfo()
                                    fracAreaLCtoMCD = 0 
                                    if (totalArea > 0) :
                                        fracAreaLCtoMCD = validArea / totalArea
                                    print('fracAreaLCtoMCD:',fracAreaLCtoMCD)  
                                    
                                    # check if we have enough valid cloud free aggregated matches
                                    if ( fracAreaLCtoMCD > minfracAreaLCtoMCD ) :
 

                                        imageMCD15 = imageMCD15.select(['satFlag','snowFlag','FparLai_QC','FparExtra_QC']) \
                                                                .addBands(getLUTout(imageMCD15)) \
                                                                 .addBands(getLUTL0XinB45(imageMCD15)) \
                                                                 .addBands(getLUTL0XinB36(imageMCD15)) \
                                                                 .addBands(getLUTinMOD09(imageMCD15)) \
                                                                 .addBands(getLUTinMOD09bands(imageMCD15)) \
                                                                 .addBands(ee.Image.pixelLonLat()) \
                                                                 .updateMask(validLCtoMCD)
        
                                        sampleData = imageMCD15.sample(projection=projectionMCD15.atScale(outputScale), numPixels=maxSample, dropNulls=True)
                                        sampleOutput = sampleData.aggregate_array('LUTout').flatten().getInfo()
                                        sampleInputB45 = sampleData.aggregate_array('LUTL0XinB45').flatten().getInfo()
                                        sampleInputB36 = sampleData.aggregate_array('LUTL0XinB36').flatten().getInfo()
                                        sampleInputMOD09 = sampleData.aggregate_array('LUTinMOD09').flatten().getInfo()
                                        sampleInputMOD09bands = sampleData.aggregate_array('LUTinMOD09bands').flatten().getInfo()
                                        sampleInputLon = sampleData.aggregate_array('longitude').flatten().getInfo()
                                        sampleInputLat = sampleData.aggregate_array('latitude').flatten().getInfo()
                                        sampleFparLai_QC = sampleData.aggregate_array('FparLai_QC').flatten().getInfo()
                                        sampleFparExtra_QC = sampleData.aggregate_array('FparExtra_QC').flatten().getInfo()

                                        if (len(sampleOutput)>0) :

                                            numsam = len(sampleOutput)
                                            cumsam = cumsam + numsam
                                            print('cumulative samples:',cumsam,'nsaminput',numsam)
                                            
                                            # write list after each landsat scene is done (comment out if it is too frequent)
                                            with open(InputFileNameLon, 'ab') as fp1:
                                                pickle.dump(sampleInputLon, fp1)
                                            with open(InputFileNameLat, 'ab') as fp1:
                                                pickle.dump(sampleInputLat, fp1)
                                            with open(InputFileNameB36, 'ab') as fp1:
                                                pickle.dump(sampleInputB36, fp1)
                                            with open(InputFileNameB45, 'ab') as fp1:
                                                pickle.dump(sampleInputB45, fp1)
                                            with open(InputFileNameMOD09, 'ab') as fp1:
                                                pickle.dump(sampleInputMOD09, fp1)
                                            with open(InputFileNameMOD09bands, 'ab') as fp1:
                                                pickle.dump(sampleInputMOD09bands, fp1)
                                            with open(OutputFileName, 'ab') as fp1:
                                                pickle.dump(sampleOutput, fp1)                                        
                                            with open(YearFileName, 'ab') as fp1:
                                                pickle.dump([year]*numsam, fp1)
                                            with open(MonthFileName, 'ab') as fp1:
                                                pickle.dump([month]*numsam, fp1)
                                            with open(DayFileName, 'ab') as fp1:
                                                pickle.dump([daystart]*numsam, fp1)
                                            with open(PathFileName, 'ab') as fp1:
                                                pickle.dump([path]*numsam, fp1 ) 
                                            with open(RowFileName, 'ab') as fp1:
                                                pickle.dump([row]*numsam, fp1)  
                                            with open(FparLai_QCName, 'ab') as fp1:
                                                pickle.dump(sampleFparLai_QC, fp1 ) 
                                            with open(FparExtra_QCName, 'ab') as fp1:
                                                pickle.dump(sampleFparExtra_QC, fp1)  



year 2013 month 6 path 6 row 22
fracAreaMCD15: 0.0
year 2013 month 6 path 8 row 26
year 2013 month 6 path 10 row 27
year 2013 month 6 path 10 row 28
fracAreaMCD15: 0.04336703958808371
year 2013 month 6 path 11 row 10
fracAreaMCD15: 0.00041663838584195105
year 2013 month 6 path 11 row 12
fracAreaMCD15: 0.0
year 2013 month 6 path 11 row 19
fracAreaMCD15: 0.0
year 2013 month 6 path 11 row 21
fracAreaMCD15: 0.0
year 2013 month 6 path 11 row 22
fracAreaMCD15: 0.012971281643266152
year 2013 month 6 path 11 row 23
fracAreaMCD15: 0.020624402203249546
year 2013 month 6 path 11 row 24
fracAreaMCD15: 0.24379043530083572
year 2013 month 6 path 11 row 25
year 2013 month 6 path 12 row 18
year 2013 month 6 path 12 row 19
fracAreaMCD15: 0.000312531954127939
year 2013 month 6 path 13 row 16
fracAreaMCD15: 0.0
year 2013 month 6 path 13 row 17
fracAreaMCD15: 0.0
year 2013 month 6 path 13 row 18
fracAreaMCD15: 0.0
year 2013 month 6 path 13 row 19
fracAreaMCD15: 0.0
year 2013 month 6 path 13 row 20
fracAre

EEException: Image.select: Parameter 'input' is required.